In [18]:
# !pip install snowflake-snowpark-python[pandas]==1.5.0
# !pip install snowflake-connector-python
# !pip install snowflake-ml-python==1.0.10
!pip install refractml

     |████████████████████████████████| 51kB 1.4MB/s eta 0:00:011
     |████████████████████████████████| 71kB 1.8MB/s eta 0:00:011
     |████████████████████████████████| 61kB 6.2MB/s  eta 0:00:01
     |████████████████████████████████| 143kB 8.8MB/s eta 0:00:01
     |████████████████████████████████| 665kB 21.3MB/s eta 0:00:01
     |████████████████████████████████| 51kB 8.5MB/s  eta 0:00:01
     |████████████████████████████████| 9.6MB 35.9MB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/f9/9b/335f9764261e915ed497fcdeb11df5dfd6f7bf257d4a6a2a686d80da4d54/requests-2.32.3-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/54/30/c2a907b9443cf42b90c17ad10c1e8fa801975f01cb9764f3f8eb8aea638b/numpy-1.26.4-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/35/f5/d0ad1a96f80962ba65e2ce1de6a1e59edecd1f0a7b55990ed208848012e0/scipy-1.13.1-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.w

You should consider upgrading via the 'pip install --upgrade pip' command.


In [ ]:
!pip install snowflake[ml]==0.8.0

In [ ]:
# !pip install snowflake-connector-python
!pip uninstall cloudpickle -y
!pip install cloudpickle==2.2.1

In [1]:
from snowflake.snowpark import Session

In [2]:
REGISTRY_DATABASE_NAME = "MODEL_REGISTRY_SNOWSIGHT"
REGISTRY_SCHEMA_NAME = "SNOWINSIGHT"

In [3]:
import configparser
def get_session():
    parser = configparser.ConfigParser()
    # Add the credential file name here
    parser.read('/notebooks/notebooks/config.ini')

    connection_params = dict(user=parser['Credentials']['user'], 
                         password=parser['Credentials']['password'], 
                         account=parser['Credentials']['account'], 
                         warehouse=parser['Credentials']['warehouse'], 
                         database=REGISTRY_DATABASE_NAME,
                         schema=REGISTRY_SCHEMA_NAME, 
                         role=parser['Credentials']['role'])
#     print("connection_params = ",  connection_params)


    session = Session.builder.configs(connection_params).create()
    return session

In [4]:
session = get_session()
# session.close()
session

In [5]:
# from snowflake.ml.registry import model_registry
from snowflake.ml.registry import Registry as model_registry

In [ ]:
!pip list | grep -i sci

In [ ]:
session.sql("LIST STAGES")

In [ ]:
# model_registry.create_model_registry(session=session, database_name=REGISTRY_DATABASE_NAME, schema_name=REGISTRY_SCHEMA_NAME)

In [6]:
registry = model_registry(session=session, database_name=REGISTRY_DATABASE_NAME, schema_name=REGISTRY_SCHEMA_NAME )

In [ ]:
# registry = model_registry.ModelRegistry(session=session, database_name=REGISTRY_DATABASE_NAME, schema_name=REGISTRY_SCHEMA_NAME)

In [ ]:
df = registry.show_models()
df
# type(df)

# Get model list from snowpark registry

In [ ]:
model_list = registry.models()
model_list

# Get model details from registry

In [7]:
# m =registry.get_model("sample_sklearn_model")
m =registry.get_model("sample_model_9")

m

# Get model versions

In [8]:
df_version = m.show_versions()
df_version

,created_on,name,comment,database_name,schema_name,module_name,is_default_version,functions,metadata,user_data
0,2024-06-04 03:51:57.033000-07:00,RUN4,test,MODEL_REGISTRY_SNOWSIGHT,SNOWINSIGHT,SAMPLE_MODEL_9,true,"[""PREDICT""]","{""metrics"": {""model_metrics"": {""score"": 96}, ""...","{""snowpark_ml_data"":{""functions"":[{""name"":""PRE..."


# Set default model version

In [ ]:
# default_version = m.default
m.default = "V1"

# Get default model version details from model-registry

In [9]:
mv = m.default
mv

In [ ]:
x

In [ ]:
#Load default vesion as python object
# clf = mv.load()

# Run prediction on scikit-learn model

In [17]:
import numpy as np
data_list = [[ 25, 185,  88,  78,  65,  81,  77,  75,  81,  63,  37,  26,   9, 10,  79,  82]]
data_array = np.asarray(data_list)
print("data_array =", data_array)

remote_prediction = mv.run(X_train.head(1), function_name="predict")
remote_prediction   # assuming test_features is Snowpark DataFrame
remote_prediction

data_array = [[ 25 185  88  78  65  81  77  75  81  63  37  26   9  10  79  82]]


/tmp/pip_packages/snowflake/snowpark/session.py:2367: UserWarning: Pandas Dataframe has non-standard index of type <class 'pandas.core.indexes.base.Index'> which will not be written. Consider changing the index to pd.RangeIndex(start=0,...,step=1) or call reset_index() to keep index as column(s)
  success, _, _, ci_output = write_pandas(


,output_feature_0
0,1.606879e+07


In [ ]:
# import cloudpickle

# model_obj = cloudpickle.load(open("/notebooks/notebooks/ml_model", "rb"))
# model_obj


In [ ]:
type(X_train)

In [ ]:
# x_train = cloudpickle.load(open("/notebooks/notebooks/x_train", "rb"))
# x_train
# type(x_train)

# Deploy model with custom class

In [ ]:
from snowflake.ml.model import custom_model

In [ ]:
class ExamplePipelineModel(custom_model.CustomModel):
#     def __init__(self, context: ModelContext) -> None:
#         super().__init__(context)
#         v = int(open(context.path('config')).read())
#         self.bias = json.loads(v)['bias']

    @custom_model.inference_api
    def run(self, data_list: list) -> pd.DataFrame:
#         features = self.context.model_ref('feature_preproc').transform(input)
#         model_output = self.context.model_ref('m2').predict(
#             self.context.model_ref('m1').predict(features)
#         )
#         return pd.DataFrame({
#             'output': model_output + self.bias})
        data_array = np.asarray(data_list)
        try:
            prediction = model.predict(data_array)
        except:
            prediction = model.predict(data_array.reshape(1, -1))
        return prediction.tolist()

In [ ]:
!pip freeze | grep cloud

# Train & register model

In [11]:
import numpy as np
import pandas as pd

from sklearn.ensemble import GradientBoostingRegressor, GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV

In [12]:
# Load the data
data = pd.read_csv("/data/fifa.csv")
data.head(10)

,short_name,nationality,overall,potential,wage_eur,value_eur,age,height_cm,weight_kg,attacking_crossing,...,mentality_penalties,mentality_composure,defending_marking,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes
0,L. Messi,Argentina,94,94,565000,95500000,32,170,72,88,...,75,96,33,37,26,6,11,15,14,8
1,Cristiano Ronaldo,Portugal,93,93,405000,58500000,34,187,83,84,...,85,95,28,32,24,7,11,15,14,11
2,Neymar Jr,Brazil,92,92,290000,105500000,27,175,68,87,...,90,94,27,26,29,9,9,15,15,11
3,J. Oblak,Slovenia,91,93,125000,77500000,26,188,87,13,...,11,68,27,12,18,87,92,78,90,89
4,E. Hazard,Belgium,91,91,470000,90000000,28,175,74,81,...,88,91,34,27,22,11,12,6,8,8
5,K. De Bruyne,Belgium,91,91,370000,90000000,28,181,70,93,...,79,91,68,58,51,15,13,5,10,13
6,M. ter Stegen,Germany,90,93,250000,67500000,27,187,85,18,...,25,70,25,13,10,88,85,88,88,90
7,V. van Dijk,Netherlands,90,91,200000,78000000,27,193,92,53,...,62,89,91,92,85,13,10,13,11,11
8,L. Modric,Croatia,90,90,340000,45000000,33,172,66,86,...,82,92,68,76,71,13,9,7,14,9
9,M. Salah,Egypt,90,90,240000,80500000,27,175,71,79,...,77,91,38,43,41,14,14,9,11,14


In [ ]:
type(y_train)

In [13]:
X = data[['age','height_cm','weight_kg','skill_dribbling','attacking_crossing','attacking_finishing','movement_acceleration','movement_sprint_speed','power_shot_power','mentality_aggression','defending_marking','defending_standing_tackle','goalkeeping_diving','goalkeeping_handling','overall','potential']]
y = data['value_eur']
ylog = np.log(y)

# %matplotlib inline
# import matplotlib.pyplot as plt
# plt.hist(ylog, bins='auto')
# plt.title("ln(value_eur)")
# plt.show()

X_train, X_test, ylog_train, ylog_test, y_train, y_test = train_test_split(X, ylog, y, test_size=0.25, random_state=4)

In [14]:
gbm_default = GradientBoostingRegressor()
gbm_default.fit(X_train, y_train)

GradientBoostingRegressor()

In [16]:
gbm_default.predict(X_train.head(1))

array([16068786.55435647])

In [ ]:
import joblib
def dump_model(model, path):
    joblib.dump(model, path)

In [ ]:
dump_model(gbm_default, "/notebooks/notebooks/ml_model_4")

# Generating model signature

In [ ]:
from snowflake.ml.model import model_signature

In [ ]:
sig = model_signature.infer_signature(
    X_train.head(),
    y_train.head().tolist(),
    input_feature_names=X_train.columns.tolist(),
    output_feature_names=['value_eur'])

In [ ]:
mv = registry.log_model(gbm_default,
                   model_name="sample_sklearn_model_5",
                   version_name="v1",
                   conda_dependencies=["scikit-learn==1.3.2", "scipy==1.13.1", "cloudpickle==2.2.1"],
                   comment="My awesome ML model",
                    python_version="3.9.19",
                   metrics={"score": 96},
                   sample_input_data=X_train.head()
                       )

In [ ]:
import joblib
def dump_model(model, path):
    joblib.dump(model, path)

In [ ]:
dump_model(gbm_default, "/notebooks/notebooks/ml_model_5")

# Create stage in snowflake-model-registry

In [ ]:
# query = """
#         create or replace stage sample_model_serving_udf_stg
#         directory = (enable = true)
#         copy_options = (on_error='skip_file')
#         """

# session.sql(query)



In [ ]:
# To see the stages
session.sql("show stages")

In [ ]:
from snowflake.snowpark.files import SnowflakeFile

In [ ]:
import cloudpickle
with SnowflakeFile.open("@STG_ML_MODELS/ml_model_2") as file:
    m= cloudpickle.load(file)
    print(type(m))

# Upload model file on ML_MODELS stage 

In [ ]:
#create the stage for storing the ML models
# session.sql("USE MODEL_REGISTRY_SNOWSIGHT;")
session.sql('CREATE OR REPLACE STAGE STG_ML_MODELS').show()

In [ ]:
X_train.to_csv("/data/X_train.csv", index = False)

In [ ]:
session.file.put(
    "/data/X_train.csv", "@STG_ML_MODELS", auto_compress=False, overwrite=True
)

In [ ]:
session.file.put(
    "/notebooks/notebooks/ml_model_5", "@STG_ML_MODELS", auto_compress=False, overwrite=True
)

In [ ]:
session.file.put(
    "/data/ml_model_5", "@STG_ML_MODELS", auto_compress=False, overwrite=True
)

# Read model file from stage

In [ ]:
session.clear_imports()
session.clear_packages()

#Register above uploded model as import of UDF
session.add_import("@STG_ML_MODELS/ml_model_2")

#map packege dependancies
session.add_packages("scikit-learn", "pandas", "cloudpickle==2.2.1")

In [ ]:
from snowflake.snowpark.types import PandasSeries, PandasDataFrame
import snowflake.snowpark.functions as F
from typing import  Optional



def read_file(filename):
    import cloudpickle
    import sys
    import os
    
    #where all imports located at
    import_dir = sys._xoptions.get("snowflake_import_directory")

    if import_dir:
        with open(os.path.join(import_dir, filename), 'rb') as file:
            m = cloudpickle.load(file)
            return m

#register UDF
@F.udf(name = 'READ_MODEL_FROM_FILE', is_permanent = True, replace = True, stage_location = '@STG_ML_MODELS')
def read_model_from_file(file_name:Optional[str]) -> str:
    import numpy as np
    
    # later we will input train data as JSON object
    # hance, we have to convert JSON object as pandas DF
    data_array = np.asarray(data_list)
    pipeline = read_file(file_name)
    print("type of pickle object =", type(pipeline))
#     dict_mod["model_obj"] = pipeline
    return "pipeline"
#     return pipeline.predict(data_array).to_list()




# @F.udf(name = 'READ_MODEL_FROM_FILE', is_permanent = True, replace = True, stage_location = '@STG_ML_MODELS')
# def read_model_from_file(file_name:Optional[str]) -> str:
#     import numpy as np
    
#     # later we will input train data as JSON object
#     # hance, we have to convert JSON object as pandas DF
#     data_array = np.asarray(data_list)
#     pipeline = read_file(file_name)
#     print("type of pickle object =", type(pipeline))
# #     dict_mod["model_obj"] = pipeline
#     return pipeline
# #     return pipeline.predict(data_array).to_list()

In [ ]:
session.sql("DESC FUNCTION READ_MODEL_FROM_FILE("ml_model")").show()

In [ ]:
model_udf = read_model_from_file("ml_model_2")
# model_udf = read_file("ml_model_2")

In [ ]:
model_udf

In [ ]:
import joblib

In [ ]:
# model_obj = joblib.load(read_model_from_file("ml_model_2"))
import os
model_file = os.path.join("/notebooks/notebooks", "ml_model_3")
model_obj = joblib.load(model_file)

In [ ]:
X_train.to_csv("/data/X_train.csv")

In [ ]:
import numpy as np

In [ ]:
data_list = [[ 25, 185,  88,  78,  65,  81,  77,  75,  81,  63,  37,  26,   9, 10,  79,  82]]
data_array = np.asarray(data_list)
print("data_array =", data_array)

remote_prediction = model_obj.run(data_array, function_name="predict")
remote_prediction   # assuming test_features is Snowpark DataFrame
remote_prediction

In [ ]:
from snowflake.snowpark import DataFrame

In [ ]:
import pandas as pd

In [ ]:
from snowflake.snowpark.types import Variant
def read_model_from_stage(session: Session, df_dict: dict) -> Variant:

    # print("file_name =", file_name)
    # session.add_import("@STG_ML_MODELS/"+file_name)
    # map packege dependancies
    # session.add_packages("scikit-learn", "pandas", "cloudpickle==2.2.1")



    # from snowflake.snowpark.types import PandasSeries, PandasDataFrame
    # import snowflake.snowpark.functions as F
    # from typing import Optional
    from snowflake.snowpark.files import SnowflakeFile
    from snowflake.ml.registry.registry import Registry

    import joblib
    df = pd.DataFrame(df_dict)
    print("df ===", df)

    model_file = '@STG_ML_MODELS/ml_model_5'
    # Specify 'mode = rb' to open the file in binary mode.
    with SnowflakeFile.open(model_file, 'rb', require_scoped_url=False) as f:
        model_obj = joblib.load(f)
        # return str(model_obj)
        # return str(model_obj)

        reg = Registry(session=session)

        mv = reg.log_model(model=model_obj,
                           model_name="sample_model",
                           comment="test",
                           version_name="run1",
                           python_version="3.9.19",
                           conda_dependencies=["scikit-learn==1.3.2"],
                           metrics={"model_metrics": {"score": 96}, "project_id": "0001", "type": "Model"},
                           sample_input_data=df
                           )

        print("model got registered successfully")
        return df_dict

In [ ]:
session.sproc.register(func=read_model_from_stage,
                           name="read_model_from_stage",
                           packages=["snowflake-snowpark-python", "snowflake-ml-python","scikit-learn", "pandas", "cloudpickle==2.2.1", "joblib"],
                           isPermanant=True,
                           stage_location="@STG_ML_MODELS",
                           replace=True,
                       )

In [ ]:
dict_df = X_train.head().to_dict()

In [ ]:
dict_df

In [ ]:
X_train.columns

In [ ]:
df = pd.DataFrame(dict_df)

In [ ]:
df.columns

In [ ]:
model_obj = session.call("read_model_from_stage", dict_df)

In [ ]:
!pip freeze | grep -i categ

In [ ]:
!pip install category_encoders

In [ ]:
import pandas as pd
import numpy as np
# import category_encoders as ce
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
import requests
# from refractml import *
# from refractml.constants import MLModelFlavours

In [ ]:
cars = pd.read_csv("/data/car_new.csv")
cars

In [ ]:
Y = cars['class_num']
X = cars.drop (['class_num'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.18, shuffle=True, random_state=25)
print (X_train.shape, X_test.shape, y_train.shape, y_test.shape)

In [ ]:
first_tree = DecisionTreeClassifier()
model=first_tree.fit(X_train, y_train)
model

In [ ]:
y_pred = model.predict(X_test)
first_tree.score(X_test, y_test)
y_prob = model.predict_proba(X_test)[:,1]

In [ ]:
model.predict(X_train.head(1))

In [ ]:
from snowflake.ml.model import model_signature

In [ ]:
y_train.head().tolist()

In [ ]:
sig = model_signature.infer_signature(
    X_train.head(),
    y_train.head().tolist(),
    input_feature_names=X_train.columns.tolist(),
    output_feature_names=['class_num'])

# 